In [513]:
import pandas as pd

# loading data keeping necesary columns
try:
    county_2022 = pd.read_excel('data/Ixodes_County_Table.xlsx', sheet_name = 3, dtype= str, names = ['FIPSCode', 'State', 'County', 'Scap_Status_2022', 'Pac_Status_2022'], header = None, skiprows = 1, usecols = [0, 1, 2, 3, 5])
    county_2021 = pd.read_excel('data/Ixodes_County_Table.xlsx', sheet_name = 4, dtype= str, names = ['FIPSCode', 'Scap_Status_2021', 'Pac_Status_2021'], header = None, skiprows = 1, usecols = [0, 3, 5])
    county_2020 = pd.read_excel('data/Ixodes_County_Table.xlsx', sheet_name = 5, dtype= str, names = ['FIPSCode', 'Scap_Status_2020', 'Pac_Status_2020'], header = None, skiprows = 1, usecols = [0, 3, 5])
    county_2019 = pd.read_excel('data/Ixodes_County_Table.xlsx', sheet_name = 6, dtype= str, names = ['FIPSCode', 'Scap_Status_2019', 'Pac_Status_2019'], header = None, skiprows = 1, usecols = [0, 3, 5])
    county_2018 = pd.read_excel('data/Ixodes_County_Table.xlsx', sheet_name = 7, dtype= str, names = ['FIPSCode', 'Scap_Status_2018', 'Pac_Status_2018'], header = None, skiprows = 1, usecols = [0, 3, 5])
    county_2016 = pd.read_excel('data/Ixodes_County_Table.xlsx', sheet_name = 8, dtype= str, names = ['FIPSCode', 'Scap_Status_2016', 'Pac_Status_2016'], header = None, skiprows = 1, usecols = [0, 3, 5])
    county_1996 = pd.read_excel('data/Ixodes_County_Table.xlsx', sheet_name = 9, dtype= str, names = ['FIPSCode', 'Scap_Status_1996', 'Pac_Status_1996'], header = None, skiprows = 1, usecols = [0, 3, 5])
    pathogen_2022 = pd.read_excel('data/Ixodes_Pathogens_County_Table.xlsx', sheet_name = 3, dtype= str, names = ['FIPSCode', 'Borrelia_burgdorferi_sensu_stricto_2022', 'Borrelia_mayonii_2022', 'Borrelia_miyamotoi_2022', 'Anaplasma_phagocytophilum_2022', 'Ehrlichia_muris_eauclairensis_2022', 'Babesia_microti_2022', 'Powassan_virus_2022'], header = None, skiprows = 1, usecols = [0, 3, 5, 7, 9, 11, 13, 15])
    pathogen_2021 = pd.read_excel('data/Ixodes_Pathogens_County_Table.xlsx', sheet_name = 4, dtype= str, names = ['FIPSCode', 'Borrelia_burgdorferi_sensu_stricto_2021', 'Borrelia_mayonii_2021', 'Borrelia_miyamotoi_2021', 'Anaplasma_phagocytophilum_2021', 'Ehrlichia_muris_eauclairensis_2021', 'Babesia_microti_2021', 'Powassan_virus_2021'], header = None, skiprows = 1, usecols = [0, 3, 5, 7, 9, 11, 13, 15])
    pathogen_2020 = pd.read_excel('data/Ixodes_Pathogens_County_Table.xlsx', sheet_name = 5, dtype= str, names = ['FIPSCode', 'Borrelia_burgdorferi_sensu_stricto_2020', 'Borrelia_mayonii_2020'], header = None, skiprows = 1, usecols = [0, 3, 5])

except:
    print("Error in loading data...")

In [514]:
# show duplicates
print(pathogen_2020[pathogen_2020['FIPSCode'].duplicated(keep=False)])

# drop duplicate county in pathogens 2020 to merge data
pathogen_2020.drop(index = 1381, inplace = True)
pathogen_2020.drop(index = 1384, inplace = True)

# check for uniqueness
print(f"FIPSCode column is unique: {pathogen_2020['FIPSCode'].is_unique}")



     FIPSCode Borrelia_burgdorferi_sensu_stricto_2020 Borrelia_mayonii_2020
1380    27131                              No records            No records
1381    27131                              No records            No records
1384    27137                              No records            No records
1385    27137                                 Present            No records
FIPSCode column is unique: True


In [515]:
# outer join all tables (some counties may not exist in some tables)
df_list = [county_2021, county_2020, county_2019, county_2018, county_2016, county_1996, pathogen_2022, pathogen_2021, pathogen_2020]
for county in df_list:
    county_2022 = county_2022.merge(county, on = ['FIPSCode'], how = 'outer', validate = 'one_to_one')
counties = county_2022

In [516]:
# validate the final table shape
print(counties.shape)
print(counties.head())

(3148, 33)
  FIPSCode State          County Scap_Status_2022 Pac_Status_2022  \
0    01001    AL  Autauga County      Established      No records   
1    01003    AL  Baldwin County      Established      No records   
2    01005    AL  Barbour County      Established      No records   
3    01007    AL     Bibb County      Established      No records   
4    01009    AL   Blount County       No records      No records   

  Scap_Status_2021 Pac_Status_2021 Scap_Status_2020 Pac_Status_2020  \
0      Established      No records      Established      No records   
1      Established      No records      Established      No records   
2      Established      No records      Established      No records   
3      Established      No records      Established      No records   
4       No records      No records       No records      No records   

  Scap_Status_2019  ... Powassan_virus_2022  \
0      Established  ...          No records   
1      Established  ...          No records   
2     

In [517]:
# a few corrections to the data

# add missing counties in AK
state_county_list = [
    {'FIPSCode': '02230', 'State': 'AK', 'County': 'Skagway'},
    {'FIPSCode': '02063', 'State': 'AK', 'County': 'Chugach'},
    {'FIPSCode': '02275', 'State': 'AK', 'County': 'Wrangell'}
]

counties = counties.append(state_county_list, ignore_index = True)

# Shannon County (46113) was renamed to Oglala Lakota County in 2014 and reassigned fips code 46102
counties.drop(counties[counties['FIPSCode'] == '46102'].index, inplace = True)
counties.loc[counties['FIPSCode'] == '46113', 'FIPSCode'] = '46102'
counties.loc[counties['FIPSCode'] == '46102', 'County'] = 'Oglala Lakota County'

# In 2013, Bedford City (51515), an independent city, merged with Bedford county (FIPS code=51019)
counties.drop(counties[counties['FIPSCode'] == '51515'].index, inplace = True)

# check that the fips code column is still unique
print(f"FIPSCode column is unique: {counties['FIPSCode'].is_unique}")

# Drop the rows with invalid FIPSCodes
counties.drop(index = [3139, 3140, 3141, 3147],  inplace = True)

# replace null and invalid status values with no records
POP_DESC = ['Established', 'Reported', 'Present', 'No Records']
for col in counties.loc[:, 'Scap_Status_2022':'Borrelia_mayonii_2020']:
    counties[col] = counties[col].apply(lambda x: 'No records' if str(x).title() not in POP_DESC else x)

# there are missing state and county data for the following indexes
state_county_dict = {
    3116:{'State': 'AK', 'County': 'Aleutians East Borough'}, 
    3117:{'State': 'AK', 'County': 'Aleutians West Census Area'}, 
    3118:{'State': 'AK', 'County': 'Anchorage Borough'}, 
    3119:{'State': 'AK', 'County': 'Bethel Census Area'}, 
    3120:{'State': 'AK', 'County': 'Bristol Bay Borough'}, 
    3121:{'State': 'AK', 'County': 'Dillingham Census Area'}, 
    3122:{'State': 'AK', 'County': 'Haines Borough'}, 
    3123:{'State': 'AK', 'County': 'Juneau Borough'}, 
    3124:{'State': 'AK', 'County': 'Kenai Peninsula Borough'}, 
    3125:{'State': 'AK', 'County': 'Ketchikan Gateway Borough'}, 
    3126:{'State': 'AK', 'County': 'Kodiak Island Borough'},
    3127:{'State': 'AK', 'County': 'Lake and Peninsula Borough'}, 
    3128:{'State': 'AK', 'County': 'Matanuska-Susitna Borough'}, 
    3129:{'State': 'AK', 'County': 'Nome Census Area'}, 
    3130:{'State': 'AK', 'County': 'Northwest Arctic Borough'}, 
    3131:{'State': 'AK', 'County': 'Prince of Wales-Outer Ketchikan Census Area'}, 
    3132:{'State': 'AK', 'County': 'Sitka Borough'}, 
    3133:{'State': 'AK', 'County': 'Skagway-Hoonah-Angoon Census Area'}, 
    3134:{'State': 'AK', 'County': 'Valdez-Cordova Census Area'}, 
    3135:{'State': 'AK', 'County': 'Wade Hampton Census Area'}, 
    3136:{'State': 'AK', 'County': 'Wrangell-Petersburg Census Area'}, 
    3137:{'State': 'AK', 'County': 'Yakutat Borough'},
    3138:{'State': 'AK', 'County': 'Yukon-Koyukuk Census Area'}, 
    3142:{'State': 'HI', 'County': 'Hawaii County'}, 
    3143:{'State': 'HI', 'County': 'Honolulu County'}, 
    3144:{'State': 'HI', 'County': 'Kalawao County'}, 
    3145:{'State': 'HI', 'County': 'Kauai County'}, 
    3146:{'State': 'HI', 'County': 'Maui County'}
}

temp_data = pd.DataFrame.from_dict(state_county_dict, orient='index')
for index, row in temp_data.iterrows():
    # Check if the index exists in the existing DataFrame
    if index in counties.index:
        # Update the corresponding row in the existing DataFrame
        counties.loc[index, 'State'] = row['State']
        counties.loc[index, 'County'] = row['County']


# check for null values across cols to ensure data is correct
for col in counties:
    print(col, counties[col].isnull().values.any())

FIPSCode column is unique: True
FIPSCode False
State False
County False
Scap_Status_2022 False
Pac_Status_2022 False
Scap_Status_2021 False
Pac_Status_2021 False
Scap_Status_2020 False
Pac_Status_2020 False
Scap_Status_2019 False
Pac_Status_2019 False
Scap_Status_2018 False
Pac_Status_2018 False
Scap_Status_2016 False
Pac_Status_2016 False
Scap_Status_1996 False
Pac_Status_1996 False
Borrelia_burgdorferi_sensu_stricto_2022 False
Borrelia_mayonii_2022 False
Borrelia_miyamotoi_2022 False
Anaplasma_phagocytophilum_2022 False
Ehrlichia_muris_eauclairensis_2022 False
Babesia_microti_2022 False
Powassan_virus_2022 False
Borrelia_burgdorferi_sensu_stricto_2021 False
Borrelia_mayonii_2021 False
Borrelia_miyamotoi_2021 False
Anaplasma_phagocytophilum_2021 False
Ehrlichia_muris_eauclairensis_2021 False
Babesia_microti_2021 False
Powassan_virus_2021 False
Borrelia_burgdorferi_sensu_stricto_2020 False
Borrelia_mayonii_2020 False


/var/folders/2n/nbh_j1612xl31cxcjhl7k73w0000gn/T/ipykernel_55949/2232719766.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  counties = counties.append(state_county_list, ignore_index = True)


In [518]:
# In my analysis I want to compare the prevelance of populations using a scale: 
# {0 : no records, 1 : reported, 2 : Established since 2022, 3 : Established since 2021, 4 : Established since 2020,
# 5 : Established since 2019, 6 : Established since 2018, 7 : Established since 2016, 8 : Established since 1996}

# if county is established then get the count of # of established (a county remains as established after the first as per guidelines set by CDC)
# otherwise see if it has any reported years
# else it has no records

counties['Established_Count'] = counties.apply(lambda row: row.str.count('Established').sum(), axis = 1)
counties['Status_Scale'] = counties.apply(lambda row: row['Established_Count'] if row['Established_Count'] > 0 else 0 if 'Reported' in row.values else -1, axis = 1)


In [519]:
# save fips as string otherwise to_excel will strip our leading zeros from fips code
counties['FIPSCode'] = counties['FIPSCode'].astype('str')
counties.to_excel('data/county_status.xlsx', index = False)
